# Segmenting & Clustering Neighborhoods in Toronto

## Part 1 of 3 - Retrieve & normalize data

**The data is available as a Wikipedia HTML page & the page source reveals a Table Structure (Column headings & data for each row)**

```
<tbody>
<tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighborhood
</th>
</tr>

................

<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td>
</tr>

<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td>
</tr>

................

</tbody>

```

In [147]:
# import the libraries
import requests                     # Import requsts library that allows us to invoke a Web service
from bs4 import BeautifulSoup       # Utilize BeautifulSoup to parse & select data 
import pandas as pd                 # Utilize pandas DataFrame to store the retrieved table from Web scraping

# retrieve data using requests.get('url')
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')


**Utilize BeautifulSoup HTML parser to retrieve the markup document & store it in a 'soup' variable**

In [345]:
# load the data into a soup variable
soup = BeautifulSoup(data.text, 'html.parser')  # Copy the HTML document to a soup variable
tbody  = soup.find('tbody')                     # Use the embedded table body (HTML tag tbody) & retrieve the table

# Optionally print the tbody to console to visualize the HTML representation of the table
#print(tbody)

**Iterate thru the body of the table & retrieve all the rows**

In [359]:
# Create a list called table[] to store the extracted values from the HTML table
table = []

# Copy the headings & data to the table
#for tr in tbody.find_all('tr'):
#    headings = [th.text for th in tr.find_all('th')]
#    table.append(headings)
#    values   = [tr.text for td in tr.find_all('td')]
#    table.append(values)

for tr in tbody.find_all('tr'):
    values   = [tr.text for td in tr.find_all('td')]  # Retrieve column values using HTML table data tag
    #values   = [tr.text for td in tr.find_all('a')]  # Retrieve column values using HTML attribute tag
    table.append(values)

# Display the number of rows retrieved from the table
print("\nTotal number of rows retrieved from the HTML table = ", len(table))

# Optionally print the raw table to the console 
#print(table)


Total number of rows retrieved from the HTML table =  288


**Cleanup the Table; Remove Empty lists, substitute newline '\n' characters with commas**

In [358]:
# Remove empty lists, tuples, zeroes, strings
table = [x for x in table if x != []]

# Print the cleaned table to the console
print("\nTotal number of rows retained from the HTML table = ", len(table))
#print(table)

# Create 
cleanTable = []

#for lst in table:
#    for element in lst:
#        if element not in cleanTable:
#                cleanTable.append(element)

for i in range(len(table)):
#    #print(table[i][0])
    cleanTable.append([str.replace(table[i][0], '\n', ',')])

# Print the cleaned table to the console
print("\nTotal number of rows in the cleaned version table = ", len(cleanTable))
#print(cleanTable)



Total number of rows retained from the HTML table =  287

Total number of rows in the cleaned version table =  287


**Copy the cleaned table (list of lists) to a DataFrame & clean up the same**

In [475]:
# Delete the DataFrame object, df
del df, df_filtered

# Instantiate a DataFrame object, df
df = pd.DataFrame()
df_filtered = pd.DataFrame()

# Parse the cleanTable & split the comma separated values & create a dataframe
for lst in cleanTable:
    df = df.append(pd.DataFrame([sub.split(",") for sub in lst]))

# Rename the Columns from default index (a number) to desired column names 1. Postcode 2. Borough 3. Neighborhood
df.rename(columns={1:'Postcode', 2:'Borough', 3:'Neighborhood'}, inplace=True)

# Drop the 1st & last columns that don't have any values
df.drop(columns=[0,4], inplace=True)

# Sort the Dataframe by Postcode, then by Borough, & by Neighborhood
df.sort_values(['Postcode', 'Borough', 'Neighborhood'], axis=0, inplace = True)

# Rest the index (it'll insert a column named 'index' & assign the indexed values to the 'index' )
df.reset_index(inplace=True)

# Drop the index column of DataFrame since they are all zeroes anyway
df.drop(columns=['index'], inplace=True)

# Drop the rows that do not have an assigned 'Borough' name
df_filtered = df[df['Borough'] != 'Not assigned'] 

print("\nTotal number of rows filtered (after removing ones with no Borough names)  ==> ", len(df_filtered))

# Print all the rows that contain 'Not assigned' (also a sanity check for any Burroughs that slipped thru)
print("\n\n== Rows with unassigned Neighborhood names ==\n\n", df_filtered[df_filtered.eq('Not assigned').any(1)])



Total number of rows filtered (after removing ones with no Borough names)  ==>  210


== Rows with unassigned Neighborhood names ==

     Postcode       Borough  Neighborhood
249      M9A  Queen's Park  Not assigned


**Fix the unassigned values, and aggregate all the Neighborhood by Postcode & Borough**  

In [478]:
# Loop through the dataframe, copy the Borough name to Neighborhood name if the latter is unassigned
for idx in df_filtered.index:
    if(df_filtered['Neighborhood'].at[idx] == 'Not assigned'):
        print("\nBorough name of the unassigned neighborhood ( at index",idx,") ==> ", df_filtered['Borough'].at[idx])
        df_filtered['Neighborhood'].at[idx] = df_filtered['Borough'].at[idx]
        print("\nNeighborhood name reset to the Borough name ( at index",idx,") ==> ", df_filtered['Neighborhood'].at[idx])


# Display the head of the dataframe (notice it's automatically sorted by 'Postcode')
#df_filtered.head()
        
#############################################################################################################
# CONSOLIDATE THE DATAFRAME BY AGGREGATING NEIGHBORHOOD VALUES, FOR EACH POSTCODE, FOLLOWED BY BOROUGHNAME
#############################################################################################################

new_df = df_filtered.groupby(['Postcode', 'Borough']).agg(lambda col: ','.join(col))

# Reset the Index
new_df.reset_index(inplace = True)

# Print the head of the new consolidated dataframe (first 12 rows)
new_df.head(12)


,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Highland Creek,Port Union,Rouge Hill"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


**Finally print the shape of the new DataFrame object (as required to show as final step)**

In [479]:
print("The Shape of the DataFrame is ==> ", new_df.shape)

The Shape of the DataFrame is ==>  (103, 3)


## Part 2 of 3 - Enrich the Dataframe & analyze data

## Part 3 of 3 - Segment / cluster, & visualize data